In [2]:
from sqlalchemy import create_engine
connection = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')
query = f'''select * 
            from sql.pokemon 
         '''
df = pd.read_sql_query(query, connection)


NameError: name 'user' is not defined